In [1]:
import glob
import os
import pathlib
from typing import List
import pandas as pd
from tqdm.auto import tqdm
from utils.pdf_to_json_row import pdf_to_dataframe_cases
from utils.landscape_word_doc import row_to_landscape_doc
import warnings

warnings.filterwarnings("ignore")   # hides every category, every module

PDF_DIR = "_MorePDFs"
OUT_DIR = "processed_documents_case_reports" 

records : List[dict] = []

pdfs = glob.glob(os.path.join(PDF_DIR, "*.pdf"))

for pdf_path in tqdm(pdfs, total=len(pdfs), desc="Processing PDFs"):
    print(f"➜  processing {pdf_path}")
    df = pdf_to_dataframe_cases(pdf_path)

    records.append(df)

    doc_name = (
        pathlib.Path(OUT_DIR) /
        f"{pathlib.Path(pdf_path).stem}_summary.docx"
    )
    
    row_to_landscape_doc(df, doc_name)
    print(f"    ↳ Word: {doc_name}")

master = pd.concat(records, ignore_index=True)

master.to_excel(os.path.join(OUT_DIR, "all_summaries.xlsx"), index=False)
master.to_csv(os.path.join(OUT_DIR, "all_summaries.csv"),  index=False)

print("\n✅  Done — individual .docx files and a master Excel/CSV saved.")

Processing PDFs:   0%|          | 0/32 [00:00<?, ?it/s]

➜  processing _MorePDFs/s12920-021-01080-0.pdf
    ↳ Word: processed_documents_case_reports/s12920-021-01080-0_summary.docx
➜  processing _MorePDFs/s12920-022-01286-w.pdf
⚠️ SPARQL failed for 'castration-resistant prostate cancer': HTTPSConnectionPool(host='query.wikidata.org', port=443): Max retries exceeded with url: /sparql (Caused by ReadTimeoutError("HTTPSConnectionPool(host='query.wikidata.org', port=443): Read timed out. (read timeout=60)"))
    ↳ Word: processed_documents_case_reports/s12920-022-01286-w_summary.docx
➜  processing _MorePDFs/s12920-025-02112-9.pdf
    ↳ Word: processed_documents_case_reports/s12920-025-02112-9_summary.docx
➜  processing _MorePDFs/s12920-020-00850-6.pdf
⚠️ SPARQL failed for 'metastatic pancreatic cancer': HTTPSConnectionPool(host='query.wikidata.org', port=443): Max retries exceeded with url: /sparql (Caused by ReadTimeoutError("HTTPSConnectionPool(host='query.wikidata.org', port=443): Read timed out. (read timeout=60)"))
    ↳ Word: processed_doc

In [ ]:
import glob
import os
import pathlib
from typing import List
import pandas as pd
from tqdm.auto import tqdm
from utils.pdf_to_json_row import pdf_to_dataframe_cases
from utils.landscape_word_doc import row_to_landscape_doc
import warnings
from io import BytesIO
import dropbox

warnings.filterwarnings("ignore")   # hides every category, every module

APP_KEY = "wdoa57gz9vdjedr"
APP_SECRET = "wonlukbqv57utfg"
REFRESH_TOKEN = "hI3rdpRqxg8AAAAAAAAAAZ7tgALECYeGg8KOKTtH9Cb4RvO1MQfVqrsDFHT2hVMD"

# Connect (you need a Dropbox API access token!)
dbx = dropbox.Dropbox(
    app_key=APP_KEY,
    app_secret=APP_SECRET,
    oauth2_refresh_token=REFRESH_TOKEN,
)

PDF_DIR = "/PDFs_case_reports"
OUT_DIR = "processed_documents_case_reports_from_dropbox" 

records : List[dict] = []

pdfs = dbx.files_list_folder(PDF_DIR).entries

for pdf_path in tqdm(pdfs, total=len(pdfs), desc="Processing PDFs"):
    print(f"➜  processing {pdf_path.name}")

    metadata, res = dbx.files_download(f"/PDFs_case_reports/{pdf_path.name}")

    pdf_stream = BytesIO(res.content)   # create in-memory file
    try:
        df = pdf_to_dataframe_cases(pdf_stream)
    except Exception as e:
        print(f"    ⚠️  Error processing {pdf_path.name}: {e}")
        continue

    records.append(df)

    doc_name = (
        pathlib.Path(OUT_DIR) /
        f"{pathlib.Path(pdf_path.name).stem}_summary.docx"
    )
    
    row_to_landscape_doc(df, doc_name)
    print(f"    ↳ Word: {doc_name}")

# master = pd.concat(records, ignore_index=True)

# master.to_excel(os.path.join(OUT_DIR, "all_summaries.xlsx"), index=False)
# master.to_csv(os.path.join(OUT_DIR, "all_summaries.csv"),  index=False)

print("\n✅  Done — individual .docx files and a master Excel/CSV saved.")


Processing PDFs:   0%|          | 0/500 [00:00<?, ?it/s]

➜  processing 12.3_Long-term_follow_up_of_patients_with_CINCA_syndrome_efficacy_and_tolerability_of_Anakinra_treatment.pdf
⚠️ SPARQL failed for 'Chronic Infantile Neurological Cutaneous and Articular (CINCA) syndrome': HTTPSConnectionPool(host='query.wikidata.org', port=443): Max retries exceeded with url: /sparql (Caused by ReadTimeoutError("HTTPSConnectionPool(host='query.wikidata.org', port=443): Read timed out. (read timeout=60)"))
    ↳ Word: processed_documents_case_reports_from_dropbox/12.3_Long-term_follow_up_of_patients_with_CINCA_syndrome_efficacy_and_tolerability_of_Anakinra_treatment_summary.docx
➜  processing 125-dihydroxyvitamin_D_and_PTHrP_mediated_malignant_hypercalcemia_in_a_seminoma.pdf
    ↳ Word: processed_documents_case_reports_from_dropbox/125-dihydroxyvitamin_D_and_PTHrP_mediated_malignant_hypercalcemia_in_a_seminoma_summary.docx
➜  processing 14-month-old_female_with_anti-MDA5_juvenile_dermatomyositis_complicated_by_liver_disease_a_case_report.pdf
⚠️ SPARQL fail

ValueError: All strings must be XML compatible: Unicode or ASCII, no NULL bytes or control characters

In [7]:
df

,Case_description,Genetic_validation,Responsible_gene,Underlying_disease,OMIM,OrphaNet,Reference_title,PubMed_ID,Single-patient case report
0,A 39-year-old gravida 1 para 1 woman was refer...,yes,FLI1,11q23 deletion syndrome (Jacobsen syndrome),,,11q23 deletion syndrome (Jacobsen syndrome) wi...,29307309,yes


In [4]:
import os, pathlib, time, warnings
from io import BytesIO
from typing import List
import pandas as pd
from tqdm.auto import tqdm
import dropbox
from dropbox.files import FileMetadata
from dropbox.exceptions import ApiError, AuthError, BadInputError

warnings.filterwarnings("ignore")

# --- CONFIG (move secrets to env vars ASAP) ---
# APP_KEY        = os.getenv("DROPBOX_APP_KEY", "YOUR_APP_KEY")
# APP_SECRET     = os.getenv("DROPBOX_APP_SECRET", "YOUR_APP_SECRET")
# REFRESH_TOKEN  = os.getenv("DROPBOX_REFRESH_TOKEN", "YOUR_REFRESH_TOKEN")

APP_KEY = "wdoa57gz9vdjedr"
APP_SECRET = "wonlukbqv57utfg"
REFRESH_TOKEN = "hI3rdpRqxg8AAAAAAAAAAZ7tgALECYeGg8KOKTtH9Cb4RvO1MQfVqrsDFHT2hVMD"

PDF_DIR = "/PDFs_case_reports"
OUT_DIR = pathlib.Path("processed_documents_case_reports_from_dropbox")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# --- CONNECT ---
dbx = dropbox.Dropbox(app_key=APP_KEY, app_secret=APP_SECRET, oauth2_refresh_token=REFRESH_TOKEN)

# --- HELPERS ---
def list_all_files(path: str):
    """Iterate all file entries (handles pagination)."""
    res = dbx.files_list_folder(path)
    for e in res.entries:
        if isinstance(e, FileMetadata):
            yield e
    while res.has_more:
        res = dbx.files_list_folder_continue(res.cursor)
        for e in res.entries:
            if isinstance(e, FileMetadata):
                yield e

def retry_download(path: str, tries=3, base_delay=0.5):
    """Simple retry for transient API errors."""
    for i in range(tries):
        try:
            return dbx.files_download(path)
        except (ApiError, BadInputError) as e:
            if i == tries - 1:
                raise
            time.sleep(base_delay * (2 ** i))

def safe_stem(name: str) -> str:
    return pathlib.Path(name).stem or "document"

# Your utilities
from utils.pdf_to_json_row import pdf_to_dataframe_cases
from utils.landscape_word_doc import row_to_landscape_doc

# --- PROCESS ---
records: List[pd.DataFrame] = []
failures: List[dict] = []

files = [e for e in list_all_files(PDF_DIR) if e.name.lower().endswith(".pdf")]

for entry in tqdm(files, total=len(files), desc="Processing PDFs"):
    name = entry.name
    full_path = f"{PDF_DIR}/{name}"

    try:
        print(f"➜  processing {name}")
        # 1) Download
        metadata, res = retry_download(full_path)

        # 2) Parse to dataframe
        pdf_stream = BytesIO(res.content)
        df = pdf_to_dataframe_cases(pdf_stream)  # should return a DataFrame (possibly with one row)

        if isinstance(df, pd.DataFrame) and not df.empty:
            records.append(df)
        else:
            raise ValueError("pdf_to_dataframe_cases returned empty/invalid DataFrame")

        # 3) Generate Word doc (don’t let this kill the run if it fails)
        try:
            doc_name = OUT_DIR / f"{safe_stem(name)}_summary.docx"
            row_to_landscape_doc(df, doc_name)
            print(f"    ↳ Word: {doc_name}")
        except Exception as e_doc:
            failures.append({"file": name, "stage": "docx", "error": repr(e_doc)})
            print(f"    ⚠️  Word export failed for {name}: {e_doc}")

    except (AuthError) as e_auth:
        # Auth errors are usually fatal — record and break
        failures.append({"file": name, "stage": "auth", "error": repr(e_auth)})
        print(f"    ❌ Auth error on {name}: {e_auth}")
        break
    except Exception as e:
        failures.append({"file": name, "stage": "process", "error": repr(e)})
        print(f"    ⚠️  Failed on {name}: {e}")
        # continue to next file
        continue

# --- ALWAYS WRITE MASTER, EVEN IF PARTIAL ---
master_xlsx = OUT_DIR / "all_summaries.xlsx"
master_csv  = OUT_DIR / "all_summaries.csv"

if records:
    master = pd.concat(records, ignore_index=True)
    master.to_excel(master_xlsx, index=False)
    master.to_csv(master_csv, index=False)
    print(f"\n✅  Master saved: {master_xlsx} and {master_csv}")
else:
    # Create an empty file with no rows, or skip saving—your call:
    pd.DataFrame().to_excel(master_xlsx, index=False)
    pd.DataFrame().to_csv(master_csv, index=False)
    print("\n⚠️  No successful records — wrote empty master files.")

# --- OPTIONAL: SAVE FAILURE LOG ---
if failures:
    failures_df = pd.DataFrame(failures)
    fail_path = OUT_DIR / "failures_log.csv"
    failures_df.to_csv(fail_path, index=False)
    print(f"⚠️  Some files failed. See: {fail_path}")
else:
    print("🎉  No failures.")

Processing PDFs:   0%|          | 0/53352 [00:00<?, ?it/s]

➜  processing 12.3_Long-term_follow_up_of_patients_with_CINCA_syndrome_efficacy_and_tolerability_of_Anakinra_treatment.pdf
    ↳ Word: processed_documents_case_reports_from_dropbox/12.3_Long-term_follow_up_of_patients_with_CINCA_syndrome_efficacy_and_tolerability_of_Anakinra_treatment_summary.docx
➜  processing 125-dihydroxyvitamin_D_and_PTHrP_mediated_malignant_hypercalcemia_in_a_seminoma.pdf
    ↳ Word: processed_documents_case_reports_from_dropbox/125-dihydroxyvitamin_D_and_PTHrP_mediated_malignant_hypercalcemia_in_a_seminoma_summary.docx
➜  processing 14-month-old_female_with_anti-MDA5_juvenile_dermatomyositis_complicated_by_liver_disease_a_case_report.pdf
⚠️ SPARQL failed for 'juvenile dermatomyositis': HTTPSConnectionPool(host='query.wikidata.org', port=443): Max retries exceeded with url: /sparql (Caused by ReadTimeoutError("HTTPSConnectionPool(host='query.wikidata.org', port=443): Read timed out. (read timeout=60)"))
    ↳ Word: processed_documents_case_reports_from_dropbox/14-

KeyboardInterrupt: 

In [2]:
import dropbox

APP_KEY = "wdoa57gz9vdjedr"

# Start the OAuth flow
auth_flow = dropbox.DropboxOAuth2FlowNoRedirect(
    consumer_key=APP_KEY,
    use_pkce=True,
    token_access_type="offline",  # 👈 this is the key
)

authorize_url = auth_flow.start()
print("1. Go to:", authorize_url)
print("2. Click 'Allow', then paste the code here.")
# auth_code = input("Enter the authorization code: ").strip()


1. Go to: https://www.dropbox.com/oauth2/authorize?response_type=code&client_id=wdoa57gz9vdjedr&token_access_type=offline&code_challenge=eEuNseEQobf3JI5NRx_inmhsj_JEQzqT1PPvYQ9GuO0&code_challenge_method=S256
2. Click 'Allow', then paste the code here.


In [3]:
# Finish the flow
oauth_result = auth_flow.finish('RglI2ZRJniQAAAAAAAAANxNuZzKlSaFqDmnm8k5EP7Y')

print("Access token:", oauth_result.access_token)   # short-lived (4h)
print("Refresh token:", oauth_result.refresh_token) # ✅ save this one

Access token: sl.u.AGCuw8p5mCs-pHYb7V4de6SgAA9qqE21eCdyv2kj5ydBnKU2NmbxlGPM1ib6zUvxNHEnXdsZ98l63D_ROZkjjNYQi54wU699hb4a_zG_d23Ax8srNWroLCJy-1vw2t9IJEtaPRMWa-nJbeE1xrJIQK9SFuoOPZiiHRUBYL4jEshiLWpTq52ZwmjfgefHv2WKQVfT8OS2JbJ7Mcq3ZaodR1zScX5AIkkc7lImh16vAfUMZsZSHogLKe3X9tu4z96k3lkSyzQubWgMupy9Y-FJX2a_LUiLFUfQoLltV23r0VEiEG-B8ZaVp2OFHmEsaS_io21rfi4JZe_i4-DshOuDT0Fl0yEQEJzg8aCKkc2GAzrsyY2K-c0NdrAHQTA_K6bPT3Rd3KHwbRr0naO_9GtnQbl4NPt0tJ6qK1THp14pyXZqB4IGb4q__Zxkisrk1jKt9aAcyw2KQTLs65UT3gfK6W0ESzRMpCB5VuPvP3FjHYtFJXVHpLaLsOVU5Nhuc81KLIFgCb1Y-4LHXguKgQCZuetLx9br5s895rA3_c0qyyEdym7NCFL6WqrUHFRQ7w0eLoIwmK4JIFZei7E-a8EWAjqyPFugP3T6uJPWe5O-8yhfopcw-c-MGazaLlR3nhDD_oNL7tjQKOahMRobS5wbgYkgAqfZNZOKmDk3AguD6_60REm58LUkRl6sBq61XDv93_O9UtFW9eVcTycXo_X8OUg1M6T1PYS4QxLT7sWNVcnOchDWi-q9cp1LdWhEpgw7tQhndk3edyhONSioUbJrQqI3bXmFHnEa_d2BxC7NvK3N_w2Jkot6vnGD7-kNy_p742PL5KXNYWgKLuVY77YKxd7SKSHvmljnev4AebQ0Ndc1R4oe8EU1sat4qh9Spf2pRYqD6DKj1HmsEEEqCs4x0M3IqfPrmJY2qLf20rpZgCexnbgFBHkQzKESDa_q75nnSoQHlBBzZ1RLXHOPtf1K0

In [7]:
import dropbox

# Connect (you need a Dropbox API access token!)
dbx = dropbox.Dropbox("sl.u.AGAxI9cqTPR-_-HuUNhKIj51wPsM0_LYz1J4vE_FTVoXEQE1Jdj9-8GhlZ-xsDgHxi6f9YjBI6kbJWPdMVxHVKn7Kx312cDQs0q8QhEFRjMCrMQKxyKrHO-o45eDfC51Zmm3Dhk-U3FE_iaiBIAobD2s7tWbllZqb7UBL8d_3gDrDxZ5ik2bKC_GnfXVf8Gb1hhSgoDeGIM37Li9NH4rXh7dlGAKV0djmRng3GPKYeZ8Jb1wmrX73Iy8D_91YGYtXvaoc2qWhblJnoJgBJfFqhOQz1mkb4eJ4ZNR6AqztvfrF8-j1BeuM2gnVVJ6oenjZHW_6SPI_l8s74HGWRWFm1Av0cAgVJWiMMakAMfg133tUhEeDg6pgInRblbM3q_jUzP7BOa_cb7MXaww75owpy9E_HO5tC3gImVtEbNF-ab35APudldGJtd5jHn9xcDPB-bt9J0G84TaskWnykMMmx6Oq1RByx7bCP6DxkGp6G_uKdfq7jQ35mDAFZ9dchWwHhEvWjg3MwyHAYbpSYxtoUGP_AFJi3ii0CaEXtueIbXQjnITns1Mg4jLGWPasnIllWEpeCldiSAKdy_W8-DybFBiKMYADKIIKdQwjo8DHd8_5Z1T5AtrIhKVbp9CLvcLM6LHIDZ3AcFrAtHfAT4qWWOSnI2hRlaxf5dZekME94gu5cwAc4s8WkRHLQoSvdMmwMOUR5CRk9U0eTWLKSH8bOGMJtfOo7hEWLtqfg5DCOqnonSZxVlop5GkNPQ6YzU6LZ2aUidHFxxSnoRvUtGltWiPLBHzkuisCyl5DKUw5rf6gAN8eJo_ZpYBGyEM8Ie-6cvmAQEtN0Vj1xUwldM6pooEcnrxEJ8GQ5_JgnbVbFzI6FuQo-GM7a7jyWPZRzARXVruJKLNUxoxa-1759WEiq5YbFyG9OtZtdPHnGS9w3Bt-Ed9jBvOJxlC6lRAvKGzI3WYdRYikqxIQNoFz-mt01qBzrExawW1KvBm5a6fa6AAqO0g8WeOJEEQg8lLDQwl91mXrkP-m5KudDMlpa6ivgfm3qvEGntgiKgcc2FzmxOknMfxrvY3EFIYKyuRZ-daVKEVqIlDfDbYt7tgLCjIa1NYGLKIsQcqTgsGEUEM0QKL2Z34lATNiZPz1i7MLViaHIXBKvN_HjSFZ2iVV7O582s6b2XeCJe4xqRGV2JzO4ndE_EaEoJIEsCfPHvZNvFyqA9ypqlULor5sdkaHluSTeSPGNVH6FOd4qiybZh7atmJeZixsOcgaJX_AujnWnAo1kaTNh80lcd5NDEYnvvBxiir")

# List files in root folder
# 


# Download a file
# metadata, res = dbx.files_download("/example.txt")
# with open("example.txt", "wb") as f:
#     f.write(res.content)

# Upload a file
# with open("local_file.txt", "rb") as f:
#     dbx.files_upload(f.read(), "/uploaded.txt", mode=dropbox.files.WriteMode.overwrite)

In [ ]:
from PyPDF2 import PdfReader
from io import BytesIO
import dropbox

# Connect (you need a Dropbox API access token!)
dbx = dropbox.Dropbox("sl.u.AGAxI9cqTPR-_-HuUNhKIj51wPsM0_LYz1J4vE_FTVoXEQE1Jdj9-8GhlZ-xsDgHxi6f9YjBI6kbJWPdMVxHVKn7Kx312cDQs0q8QhEFRjMCrMQKxyKrHO-o45eDfC51Zmm3Dhk-U3FE_iaiBIAobD2s7tWbllZqb7UBL8d_3gDrDxZ5ik2bKC_GnfXVf8Gb1hhSgoDeGIM37Li9NH4rXh7dlGAKV0djmRng3GPKYeZ8Jb1wmrX73Iy8D_91YGYtXvaoc2qWhblJnoJgBJfFqhOQz1mkb4eJ4ZNR6AqztvfrF8-j1BeuM2gnVVJ6oenjZHW_6SPI_l8s74HGWRWFm1Av0cAgVJWiMMakAMfg133tUhEeDg6pgInRblbM3q_jUzP7BOa_cb7MXaww75owpy9E_HO5tC3gImVtEbNF-ab35APudldGJtd5jHn9xcDPB-bt9J0G84TaskWnykMMmx6Oq1RByx7bCP6DxkGp6G_uKdfq7jQ35mDAFZ9dchWwHhEvWjg3MwyHAYbpSYxtoUGP_AFJi3ii0CaEXtueIbXQjnITns1Mg4jLGWPasnIllWEpeCldiSAKdy_W8-DybFBiKMYADKIIKdQwjo8DHd8_5Z1T5AtrIhKVbp9CLvcLM6LHIDZ3AcFrAtHfAT4qWWOSnI2hRlaxf5dZekME94gu5cwAc4s8WkRHLQoSvdMmwMOUR5CRk9U0eTWLKSH8bOGMJtfOo7hEWLtqfg5DCOqnonSZxVlop5GkNPQ6YzU6LZ2aUidHFxxSnoRvUtGltWiPLBHzkuisCyl5DKUw5rf6gAN8eJo_ZpYBGyEM8Ie-6cvmAQEtN0Vj1xUwldM6pooEcnrxEJ8GQ5_JgnbVbFzI6FuQo-GM7a7jyWPZRzARXVruJKLNUxoxa-1759WEiq5YbFyG9OtZtdPHnGS9w3Bt-Ed9jBvOJxlC6lRAvKGzI3WYdRYikqxIQNoFz-mt01qBzrExawW1KvBm5a6fa6AAqO0g8WeOJEEQg8lLDQwl91mXrkP-m5KudDMlpa6ivgfm3qvEGntgiKgcc2FzmxOknMfxrvY3EFIYKyuRZ-daVKEVqIlDfDbYt7tgLCjIa1NYGLKIsQcqTgsGEUEM0QKL2Z34lATNiZPz1i7MLViaHIXBKvN_HjSFZ2iVV7O582s6b2XeCJe4xqRGV2JzO4ndE_EaEoJIEsCfPHvZNvFyqA9ypqlULor5sdkaHluSTeSPGNVH6FOd4qiybZh7atmJeZixsOcgaJX_AujnWnAo1kaTNh80lcd5NDEYnvvBxiir")

for entry in dbx.files_list_folder("/PDFs_case_reports").entries:
    print(entry.name)
    # Download a PDF from Dropbox
    metadata, res = dbx.files_download(f"/PDFs_case_reports/{entry.name}")

    pdf_stream = BytesIO(res.content)   # create in-memory file
    reader = PdfReader(pdf_stream)

    for page in reader.pages:
        print(page.extract_text())

    break


12.3_Long-term_follow_up_of_patients_with_CINCA_syndrome_efficacy_and_tolerability_of_Anakinra_treatment.pdf
BioMed Central
Page 1 of 1
(page number not for citation purposes)Pediatric Rheumatology
Open Access Oral presentation
12.3 Long-term follow up of pa tients with CINCA syndrome: 
efficacy and tolerability of Anakinra treatment
GP a l o n i *1, M Gattorno2, M Alessio3, D Rigante4, M Cattalini5, F Zulian6 and 
LL e p o r e1
Address: 1Institute of Child Health IRCCS Burlo Garofolo, University of Trieste, Italy, 2II Division of Pediatrics, 'G. Gaslini' Institute for Children, 
Genoa, Italy, 3Department of Pediatrics, Federico II Hospital, Naples, 4Department of Pediatric Sciences, Catholic University of Sacred Heart, 
Rome, Italy, 5Department of Rheumatology and Clinic al Immunology, Br escia, Italy and 6Pediatric Rheumatology Unit, Department of Pediatrics, 
Padova, Italy
* Corresponding author    
We report the long-term follow up of 15 Italian patients
affected by Chronic Infanti

In [3]:
row_to_landscape_doc(df, doc_name)

In [2]:
df

,Case_description,Genetic_validation,Responsible_gene,Underlying_disease,OMIM,OrphaNet,Reference_title,PubMed_ID
0,The proband was an 18 year old male from South...,yes,LTBP2,microspherophakia,OMIM:251750,Orphanet:1340,Compound heterozygous mutations in the LTBP2 g...,34654491


In [4]:
from utils.extract_pdf_tables import extract_tables  # custom utility

pdf = 'pdfs/founder_mennonite.pdf'
tables = extract_tables(pdf)
tables

[0                               3 )s(ecnerefeR  \
 0                          ecnerefeR secneuqeS   
 1   )ecnerefnoC yticinhtE cfiic ro noigeR epS(   
 2                            ycneuqerF reirraC   
 3  noitroporP cinegohtaP ni stnairaV 2 eneG fo   
 4           nietorP detciderP )1 egnahC sailA(   
 5              egnahC editoelcuN )1 AND sailA(   
 6                                          IOM   
 7                                     redrosiD   
 8                                         eneG   
 
 0                     re grebneffuP & ssuartS ]9002[  \
 0                                        6.610000_MN   
 1                 dnalrevaeW( )eladfforG etinonneM &   
 2                                            nwonknU   
 3                                            4 %001<   
 4                                                 --   
 5                           )G>A03-4SVI( G>A03-782.c   
 6                                                 RA   
 7  -lyca niahc-muideM esanegord

In [5]:
from docling.document_converter import DocumentConverter     # the code you patched

conv      = DocumentConverter()
md_main   = conv.convert(str(pdf)).document.export_to_markdown()

md_main

/Users/timhulshof/.pyenv/versions/3.12.3/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/timhulshof/.pyenv/versions/3.12.3/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/timhulshof/.pyenv/versions/3.12.3/lib/python3.12/site-packages/docling/pipeline/standard_pdf_pipeline.py:262: RuntimeWarning: Mean of empty slice
  np.nanmean(
/Users/timhulshof/.pyenv/versions/3.12.3/lib/python3.12/site-packages/docling/pipeline/standard_pdf_pipeline.py:267: RuntimeWarning: Mean of empty slice
  np.nanmean(


"<!-- image -->\n\nNLM Citation: Wallace SE, Gillentine MA. Resources for Genetics Professionals - Genetic Disorders Associated with Founder Variants Common in the Mennonite Population. 2023 Jun 1. In: Adam MP, Feldman J, Mirzaa GM, et al., editors. GeneReviews ® [Internet]. Seattle (WA): University of Washington, Seattle; 1993-2025. Bookshelf URL: https://www.ncbi.nlm.nih.gov/books/\n\n<!-- image -->\n\n## Resources for Genetics Professionals - Genetic Disorders Associated with Founder Variants Common in the Mennonite Population\n\nStephanie E Wallace, MD 1,2  and Madelyn A Gillentine, PhD 3,4\n\nCreated: June 1, 2023.\n\nA founder variant is a pathogenic variant observed at high frequency in a specific population due to the presence of the variant in a single ancestor or small number of ancestors. The presence of a founder variant can affect the approach to molecular genetic testing. When one or more founder variants account for a large percentage of all pathogenic variants found in 

In [2]:
tables[0]

,3 )s(ecnerefeR,re grebneffuP & ssuartS ]9002[,raliugA-zurC ]7102[ la te,te grebneerG ]3991[ la,re grebneffuP & ssuartS ]9002[,te semarahC ]8002[ la,"la te -srednuaS te arumakaN namlluP ]4102[ ,]2102[ la",re grebneffuP & ssuartS ]9002[,,nesnaH-hceB ]8991[ la te,la te ]3002[ idaD,& ]9002[ nosredneH rabnA,re ]a2102[ grebneffuP la te
0,ecnerefeR secneuqeS,6.610000_MN,1.454873100_MN 1.383563100_PN,6.874000_MN 3.964000_PN,4.349030_MN 2.502211_PN,4.184800_GN,4.150000_MN 3.240000_PN,4.907000_MN 1.007000_PN,1.856073100_MN 1.785753100_PN,4.381500_MN 2.471500_PN,6.237000_MN 1.327000_PN,4.294000_MN 3.384000_PN,5.508300_MN 1.697300_PN
1,)ecnerefnoC yticinhtE cfiic ro noigeR epS(,dnalrevaeW( )eladfforG etinonneM &,")ocixeM etinonneM ognaruD( ,etatS","etinonneM ,abotinaM( )adanaC",dnalrevaeW( )eladfforG etinonneM &,"etinonneM ,abotinaM( )adanaC",etinonneM )adanaC(,",dnalrevaeW( & etinonneM ,eladfforG )reffuatS",dnalrevaeW( )eladfforG etinonneM &,etinonneM,etinonneM,etinonneM,)ainavlysnneP( etinonneM
2,ycneuqerF reirraC,nwonknU,nwonknU,52/1,nwonknU,AN,nwonknU,ot 01/1 6/1,nwonknU,AN,nwonknU,nwonknU,15/1
3,noitroporP cinegohtaP ni stnairaV 2 eneG fo,4 %001<,5 %001~,4 %001<,5 %001~,%001<,%08~,5 %001~,4 %001<,%08~,5 %001~,4 %001<,5 %001~
4,nietorP detciderP )1 egnahC sailA(,--,reT4943nlG.p,psA433ylG.p,led442grA_032ueL.p,--,psA7602alA.p,nsA834ryT.p,grA784prT.p,11reTsforP6501ueL.p,reT86grA.p,led805ehP.p,grA821ylG.p
5,egnahC editoelcuN )1 AND sailA(,)G>A03-4SVI( G>A03-782.c,T>C08401.c,A>G1001.c,54led337_986.c,1 RTU'5 & A1 retomorp led,A>C0026.c,A>T2131.c,C>T9931.c )C>T9541(,Cpud6613.c,T>C202.c,TTCled3251_1251.c,C>G283.c
6,IOM,RA,RA,RA,RA,DA,RA,RA,RA,LX,RA,RA,RA
7,redrosiD,-lyca niahc-muideM esanegordyhed A emyzneoc yc...,emordnys mortslA,aisatahpsohpopyH,aimena )001162 citsalbolageM MIMO(,snoitidnoc detaicossa-CPA sisopylop,aisatceignalet-aixatA,eniru purys esaesid elpaM,ycneicfied esadinitoiB,yranoitats ssendnilb latinegnoC thgin,MIMO( 91 epyt )716516 DICS,sisorbfi citsyC,MIMO( ytilibasid ylahpecnessil tnairav lautcel...
8,eneG,MDACA,1SMLA,LPLA,NMA,CPA,MTA,AHDKCB,DTB,F1ANCAC,D3DC,RTFC,DDARC
